In [1]:
#importing libraries
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import os
import keras

In [2]:
# There are two classes of interests among the images in dataset
cls_of_int = ['Covid', 'Normal']

In [3]:
#setting the path variables for train, test and validation data
basePath = "Data"
trainPath = os.path.join(basePath,"train")
testPath = os.path.join(basePath,"test")
validPath = os.path.join(basePath,"valid")

trainCovid = os.path.join(trainPath,"Covid")
trainNormal = os.path.join(trainPath,"Normal")

testCovid = os.path.join(testPath,"Covid")
testNormal = os.path.join(testPath,"Normal")

validCovid = os.path.join(basePath,"valid","Covid")
validNormal = os.path.join(basePath,"valid","Normal")

In [4]:
# using a data generator for data augmentation, the volume of training data is increased using data augmentation techniques
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1.0/255, # Normalizing the image
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip = False,
    fill_mode='nearest',
    featurewise_center = True,
    featurewise_std_normalization = True)

In [5]:
# The data generator for validation data, normalizing of the image is done
valid_data_generator = ImageDataGenerator(rescale = 1./255)

In [6]:
# The data generator for test data, normalizing of the image is done
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [7]:
# Read data from directory for training data
train_generator = train_data_generator.flow_from_directory(
    directory=trainPath,
    target_size=(224, 224),# resizing the image
    color_mode="rgb",
    batch_size=32,
    class_mode="binary", # class mode is binary as there are only 2 classes
    shuffle=True,
    seed=42
)

Found 11047 images belonging to 2 classes.


In [8]:
# Read data from directory for validation data
valid_generator = valid_data_generator.flow_from_directory(
    directory= validPath,
    target_size=(224, 224),# resizing the image
    color_mode="rgb",
    batch_size=32,
    class_mode="binary", # class mode is binary as there are only 2 classes
    shuffle=True,
    seed=42
)

Found 2208 images belonging to 2 classes.


In [9]:
# Read data from directory for testing data
test_generator = test_data_generator.flow_from_directory(
    directory= testPath,
    target_size=(224, 224),# resizing the image
    color_mode="rgb",
    batch_size=11,
    class_mode="binary", # class mode is binary as there are only 2 classes
    shuffle=False,
    seed=42
)

Found 2761 images belonging to 2 classes.


In [20]:
# Downloading the Xception model.
pretrained_model = tf.keras.applications.Xception(
        weights='imagenet',
        include_top=False ,
        input_shape=(224,224,3)
    )
pretrained_model.trainable = False

# Added Pooling layer and a sigmoid activation layer.    
model = tf.keras.Sequential([ 
        pretrained_model,   
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [21]:
# Compiling the model with Adam optimizer and binary cross entropy loss function
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = opt ,
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [17]:
# To display the summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 20,865,578
Trainable params: 4,098
Non-trainable params: 20,861,480
_________________________________________________________________


In [18]:
# specifying the batch sizes
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n//train_generator.batch_size

In [22]:
# Fitting the model with 20 epochs
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

Epoch 1/20
345/345 [==============================] - 1321s 4s/step - loss: 0.4331 - accuracy: 0.8064 - val_loss: 0.3404 - val_accuracy: 0.8573
Epoch 2/20
345/345 [==============================] - 1226s 4s/step - loss: 0.3150 - accuracy: 0.8706 - val_loss: 0.3557 - val_accuracy: 0.8501
Epoch 3/20
345/345 [==============================] - 1276s 4s/step - loss: 0.2995 - accuracy: 0.8782 - val_loss: 0.3039 - val_accuracy: 0.8741
Epoch 4/20
345/345 [==============================] - 1156s 3s/step - loss: 0.2920 - accuracy: 0.8850 - val_loss: 0.3091 - val_accuracy: 0.8736
Epoch 5/20
345/345 [==============================] - 1131s 3s/step - loss: 0.2799 - accuracy: 0.8875 - val_loss: 0.2742 - val_accuracy: 0.8909
Epoch 6/20
345/345 [==============================] - 1106s 3s/step - loss: 0.2733 - accuracy: 0.8961 - val_loss: 0.3189 - val_accuracy: 0.8673
Epoch 7/20
345/345 [==============================] - 2098s 6s/step - loss: 0.2611 - accuracy: 0.8947 - val_loss: 0.3001 - val_accuracy:

In [23]:
model.evaluate(valid_generator,steps=STEP_SIZE_VALID)

69/69 [==============================] - 187s 3s/step - loss: 0.2506 - accuracy: 0.8990


[0.25055280327796936, 0.8990036249160767]

In [ ]:
model.evaluate(test_generator)

251/251 [==============================] - 8s 28ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [ ]:
score = model.evaluate(valid_generator,batch_size=32)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

69/69 [==============================] - 12s 179ms/step - loss: 0.2574 - accuracy: 0.8958
Accuracy: 89.58%
Loss:  0.25741082429885864


In [ ]:
score = model.evaluate(test_generator,batch_size=11)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

251/251 [==============================] - 7s 27ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Accuracy: 0.00%
Loss:  0.0


In [ ]:
model.save('imagegeneratormodelxc')

INFO:tensorflow:Assets written to: imagegeneratormodelxc\assets


D:\Users\Ajay\anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [ ]:
loaded_model = keras.models.load_model('imagegeneratormodelxc')

In [ ]:
val_score = loaded_model.evaluate(valid_generator)
print("Accuracy: {:.2f}%".format(val_score[1] * 100))

69/69 [==============================] - 17s 196ms/step - loss: 0.2574 - accuracy: 0.8958
Accuracy: 89.58%


In [ ]:
test_score = loaded_model.evaluate(test_generator)
print("Accuracy: {:.2f}%".format(test_score[1] * 100))

251/251 [==============================] - 20s 79ms/step - loss: 0.2775 - accuracy: 0.8877
Accuracy: 88.77%
